Dataset: https://www.kaggle.com/competitions/fake-news/data

In [67]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

nltk.download('stopwords')

df = pd.read_csv('/content/train.csv')
df = df.dropna()  # Drop NaN values

x = df.drop('label', axis=1)
y = df['label']

voc_size = 5000  # Vocabulary Size
msgs = x.copy()
msgs.reset_index(inplace=True)

ps = PorterStemmer()
corpus = []

for i in range(0, len(msgs)):
    review = re.sub('[^a-zA-Z]', ' ', msgs['title'][i])  # Keeping only alphabets
    review = review.lower()  # Converting alphabets to lowercase
    review = review.split()  # Splitting them and storing string as a list of words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # If the word is not a stopword, we will keep it after stemming it
    review = ' '.join(review)
    corpus.append(review)

onehot_repr = [one_hot(words, voc_size) for words in corpus]

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_9 (LSTM)               (None, 100)               56400     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
x_final = embedded_docs
y_final = y.values.astype(int)  # Convert y to array of integers

x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 67ms/step - loss: 0.3313 - accuracy: 0.8419 - val_loss: 0.1953 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1439 - accuracy: 0.9456 - val_loss: 0.2034 - val_accuracy: 0.9137
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1039 - accuracy: 0.9624 - val_loss: 0.2080 - val_accuracy: 0.9180
Epoch 4/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0790 - accuracy: 0.9718 - val_loss: 0.2491 - val_accuracy: 0.9181
Epoch 5/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0594 - accuracy: 0.9807 - val_loss: 0.2596 - val_accuracy: 0.9117
Epoch 6/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0446 - accuracy: 0.9855 - val_loss: 0.3163 - val_accuracy: 0.9135
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0346 - accuracy: 0.9887 - val_loss: 0.3572 - val_accuracy: 0.9112

In [70]:

y_pred_prob = model.predict(x_test)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

189/189 [==============================] - 1s 3ms/step
[[3105  314]
 [ 224 2392]]
0.9108533554266777


Trying adding dropout layer to add regularization and prevent overfitting

In [74]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

embedding_vector_features = 40

model = Sequential()

# Embedding layer
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))  # Dropout regularization to prevent overfitting

# LSTM layer
model.add(LSTM(100, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))  # Dropout regularization

# Dense layer
model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model training
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, callbacks=[early_stop])


Epoch 1/100
192/192 [==============================] - 32s 145ms/step - loss: 0.6376 - accuracy: 0.7753 - val_loss: 0.3332 - val_accuracy: 0.8900
Epoch 2/100
192/192 [==============================] - 20s 104ms/step - loss: 0.2872 - accuracy: 0.9176 - val_loss: 0.2812 - val_accuracy: 0.9152
Epoch 3/100
192/192 [==============================] - 19s 98ms/step - loss: 0.2168 - accuracy: 0.9450 - val_loss: 0.2819 - val_accuracy: 0.9082
Epoch 4/100
192/192 [==============================] - 20s 102ms/step - loss: 0.1904 - accuracy: 0.9509 - val_loss: 0.2520 - val_accuracy: 0.9155
Epoch 5/100
192/192 [==============================] - 18s 93ms/step - loss: 0.1648 - accuracy: 0.9620 - val_loss: 0.2589 - val_accuracy: 0.9143
Epoch 6/100
192/192 [==============================] - 20s 106ms/step - loss: 0.1455 - accuracy: 0.9689 - val_loss: 0.2799 - val_accuracy: 0.9138
Epoch 7/100
192/192 [==============================] - 20s 103ms/step - loss: 0.1356 - accuracy: 0.9705 - val_loss: 0.2642 - v

In [75]:
x_final = embedded_docs
y_final = y.values.astype(int)  # Convert y to array of integers

x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 18s 96ms/step - loss: 0.1646 - accuracy: 0.9597 - val_loss: 0.2483 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 17s 89ms/step - loss: 0.1498 - accuracy: 0.9649 - val_loss: 0.2568 - val_accuracy: 0.9158
Epoch 3/10
192/192 [==============================] - 17s 89ms/step - loss: 0.1366 - accuracy: 0.9706 - val_loss: 0.2656 - val_accuracy: 0.9173
Epoch 4/10
192/192 [==============================] - 20s 102ms/step - loss: 0.1213 - accuracy: 0.9762 - val_loss: 0.2652 - val_accuracy: 0.9140
Epoch 5/10
192/192 [==============================] - 17s 89ms/step - loss: 0.1183 - accuracy: 0.9764 - val_loss: 0.3108 - val_accuracy: 0.9100
Epoch 6/10
192/192 [==============================] - 17s 91ms/step - loss: 0.1107 - accuracy: 0.9787 - val_loss: 0.2725 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 19s 99ms/step - loss: 0.1072 - accuracy: 0.9806 - val_loss: 0.3077 - val_accuracy

In [76]:

y_pred_prob = model.predict(x_test)
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

189/189 [==============================] - 1s 7ms/step
[[3044  375]
 [ 193 2423]]
0.9058823529411765


Dealing with it: We are splitting the data after using the embedding which might lead to data leakage.

In [81]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Load the dataset
df = pd.read_csv('/content/train.csv')
df = df.dropna()  # Drop NaN values

# Split into features and labels
x = df.drop('label', axis=1)
y = df['label']

# Split into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Perform text preprocessing
nltk.download('stopwords')
ps = PorterStemmer()
corpus = []
for i in range(len(x_train)):
    review = re.sub('[^a-zA-Z]', ' ', x_train.iloc[i]['title'])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Vocabulary size
voc_size = 5000

# Perform one-hot encoding
onehot_repr = [one_hot(words, voc_size) for words in corpus]

# Pad sequences
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100, recurrent_dropout=0.3, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Model training
model.fit(embedded_docs, y_train, validation_split=0.2, epochs=100, batch_size=64, callbacks=[early_stop])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/100
154/154 [==============================] - 27s 160ms/step - loss: 0.6959 - accuracy: 0.7557 - val_loss: 0.3629 - val_accuracy: 0.8804
Epoch 2/100
154/154 [==============================] - 16s 105ms/step - loss: 0.3032 - accuracy: 0.9166 - val_loss: 0.2955 - val_accuracy: 0.9069
Epoch 3/100
154/154 [==============================] - 15s 95ms/step - loss: 0.2350 - accuracy: 0.9398 - val_loss: 0.2824 - val_accuracy: 0.9122
Epoch 4/100
154/154 [==============================] - 14s 89ms/step - loss: 0.1961 - accuracy: 0.9527 - val_loss: 0.3209 - val_accuracy: 0.8918
Epoch 5/100
154/154 [==============================] - 14s 90ms/step - loss: 0.1737 - accuracy: 0.9599 - val_loss: 0.2682 - val_accuracy: 0.9139
Epoch 6/100
154/154 [==============================] - 14s 89ms/step - loss: 0.1544 - accuracy: 0.9654 - val_loss: 0.2622 - val_accuracy: 0.9106
Epoch 7/100
154/154 [==============================] - 14s 88ms/step - loss: 0.1414 - accuracy: 0.9709 - val_loss: 0.2795 - val_

In [95]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_test_list = y_test.values.tolist()

y_pred_list = [int(val) for val in y_pred]

cm = confusion_matrix(y_test_list, y_pred_list)
print(cm)

accuracy = accuracy_score(y_test_list, y_pred_list)

[[3044  375]
 [ 193 2423]]


Doing hyperparameter tuning with the help of keras-tuner

In [97]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.5 MB/s eta 0:00:00


In [99]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow import keras
import tensorflow.keras.layers as layers
import keras_tuner as kt

In [101]:
# Load the train dataset
train_df = pd.read_csv('/content/train.csv')

# Drop NaN values
train_df = train_df.dropna()

# Split into features and labels
x_train = train_df.drop('label', axis=1)
y_train = train_df['label']

In [102]:
# Perform text preprocessing
nltk.download('stopwords')
ps = PorterStemmer()
corpus_train = []
for i in range(len(x_train)):
    review = re.sub('[^a-zA-Z]', ' ', x_train.iloc[i]['title'])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus_train.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Vocabulary size
voc_size = 5000

# Perform one-hot encoding
onehot_repr_train = [keras.preprocessing.text.one_hot(words, voc_size) for words in corpus_train]

# Pad sequences
sent_length = 20
embedded_docs_train = keras.preprocessing.sequence.pad_sequences(onehot_repr_train, padding='pre', maxlen=sent_length)

In [106]:
# Define the hypermodel
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Embedding(voc_size, 40, input_length=sent_length))

    # Tune the dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)
    model.add(layers.Dropout(hp_dropout))

    # Tune the number of LSTM units
    hp_units = hp.Int('units', min_value=50, max_value=150, step=10)
    model.add(layers.LSTM(units=hp_units))

    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [107]:
# Define the tuner
tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=10, factor=3, directory='kt_dir', project_name='my_model')

In [108]:
# Perform hyperparameter tuning
tuner.search(embedded_docs, y, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the model
model.fit(embedded_docs, y, epochs=10, validation_split=0.2)

Trial 30 Complete [00h 00m 38s]
val_accuracy: 0.5759183764457703

Best val_accuracy So Far: 0.5759183764457703
Total elapsed time: 00h 14m 17s
Epoch 1/10
307/307 [==============================] - 14s 39ms/step - loss: 0.6856 - accuracy: 0.5651 - val_loss: 0.6831 - val_accuracy: 0.5759
Epoch 2/10
307/307 [==============================] - 3s 9ms/step - loss: 0.6720 - accuracy: 0.5810 - val_loss: 0.6904 - val_accuracy: 0.5563
Epoch 3/10
307/307 [==============================] - 4s 13ms/step - loss: 0.6011 - accuracy: 0.6795 - val_loss: 0.7401 - val_accuracy: 0.5069
Epoch 4/10
307/307 [==============================] - 2s 7ms/step - loss: 0.5292 - accuracy: 0.7320 - val_loss: 0.8220 - val_accuracy: 0.4882
Epoch 5/10
307/307 [==============================] - 2s 7ms/step - loss: 0.4724 - accuracy: 0.7761 - val_loss: 0.9004 - val_accuracy: 0.5057
Epoch 6/10
307/307 [==============================] - 2s 7ms/step - loss: 0.4276 - accuracy: 0.7959 - val_loss: 0.9651 - val_accuracy: 0.5000
Ep

In [115]:
# Preprocess test data
corpus_test = []
for i in range(len(x_test)):
    review = re.sub('[^a-zA-Z]', ' ',str( x_test[i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus_test.append(review)

onehot_repr_test = [one_hot(words, voc_size) for words in corpus_test]
embedded_docs_test = pad_sequences(onehot_repr_test, padding='pre', maxlen=sent_length)


In [116]:
# Evaluate the model on the test data
import numpy as np
y_pred_prob = model.predict(embedded_docs_test)
y_pred = np.round(y_pred_prob).astype(int)

# Calculate accuracy and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(cm)


189/189 [==============================] - 1s 2ms/step
Accuracy: 0.49063794531897265
Confusion Matrix:
[[1558 1861]
 [1213 1403]]
